In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [4]:
my_index = "network_weather_2-*"

my_query = \
{
  "size": 10000,
  "sort": [
    {
      "timestamp": {
        "order": "desc",
        "unmapped_type": "boolean"
      }
    }
  ],
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "analyze_wildcard": True,
          "lowercase_expanded_terms": False,
          "query": "*"
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "match": {
                  "srcSite": {
                    "query": "Australia-ATLAS",
                    "type": "phrase"
                  }
                }
              }
            },
            {
              "query": {
                "match": {
                  "destSite": {
                    "query": "SFU-LCG2",
                    "type": "phrase"
                  }
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1454306400000,
                  "lte": 1456812000000,
                  "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      }
    }
  },
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "timestamp",
        "interval": "1d",
        "time_zone": "America/Chicago",
        "min_doc_count": 0,
        "extended_bounds": {
          "min": 1454306400000,
          "max": 1456812000000
        }
      }
    }
  },
  "fields": [
    "*",
    "_source"
  ],
  "script_fields": {
    "throughput [Gbps]": {
      "script": "doc['throughput'].value/1073741824.0",
      "lang": "expression"
    }
  },
  "fielddata_fields": [
    "timestamp"
  ]
}

In [5]:
response = es.search(index=my_index, body=my_query, request_timeout=600)

In [6]:
for key in response.keys():
    print(key)

hits
took
aggregations
_shards
timed_out


In [7]:
response

{'_shards': {'failed': 5,
  'failures': [{'index': 'network_weather_2-2016.1.16',
    'node': 'nqR3_gT8SCapJkludZSKiA',
    'reason': {'caused_by': {'reason': 'Field [throughput] used in expression does not exist in mappings',
      'type': 'script_exception'},
     'reason': "Error during search with inline script [doc['throughput'].value/1073741824.0] using lang [expression]",
     'type': 'script_exception'},
    'shard': 0}],
  'successful': 890,
  'total': 895},
 'aggregations': {'2': {'buckets': [{'doc_count': 527,
     'key': 1454306400000,
     'key_as_string': '2016-02-01T00:00:00.000-06:00'},
    {'doc_count': 571,
     'key': 1454392800000,
     'key_as_string': '2016-02-02T00:00:00.000-06:00'},
    {'doc_count': 754,
     'key': 1454479200000,
     'key_as_string': '2016-02-03T00:00:00.000-06:00'},
    {'doc_count': 746,
     'key': 1454565600000,
     'key_as_string': '2016-02-04T00:00:00.000-06:00'},
    {'doc_count': 1029,
     'key': 1454652000000,
     'key_as_string':

In [13]:
import pandas as pd

In [56]:
latency_rawdata = pd.DataFrame(columns=['timestamp','datetime','delay_mean','delay_median','delay_sd'])
packet_loss_rate_rawdata = pd.DataFrame(columns=['timestamp','datetime','packet_loss'])
throughput_rawdata = pd.DataFrame(columns=['timestamp','datetime','throughput'])

In [57]:
row = 0
for hit in response['hits']['hits']:
    if hit['_type'] == 'latency':
        item = hit['_source']
        dt = datetime.fromtimestamp(int(hit['fields']['timestamp'][0]/1000))
        latency_rawdata.loc[row] = [dt, item['timestamp'], item['delay_mean'], item['delay_median'], item['delay_sd']]
        row += 1

In [58]:
latency_rawdata = latency_rawdata.drop_duplicates()
latency_rawdata.head(10)

,timestamp,datetime,delay_mean,delay_median,delay_sd
0,2016-03-01 00:00:00,2016-03-01T06:00:00,80.926803,80.9,0.234238
46,2016-02-29 22:05:00,2016-03-01T04:05:00,82.945767,82.9,0.739154
53,2016-02-29 21:30:00,2016-03-01T03:30:00,86.296000,86.3,0.078000
72,2016-02-29 19:55:00,2016-03-01T01:55:00,77.228367,77.4,0.262130
90,2016-02-29 18:25:00,2016-03-01T00:25:00,81.340833,81.5,0.483414
108,2016-02-29 16:55:00,2016-02-29T22:55:00,81.415167,81.4,0.080125
116,2016-02-29 16:15:00,2016-02-29T22:15:00,81.575417,81.6,0.067298
134,2016-02-29 14:45:00,2016-02-29T20:45:00,82.651367,82.6,0.077748
141,2016-02-29 14:10:00,2016-02-29T20:10:00,82.276722,82.3,0.082039
153,2016-02-29 13:10:00,2016-02-29T19:10:00,83.219444,83.2,0.091285


In [59]:
latency_rawdata.tail(10)

,timestamp,datetime,delay_mean,delay_median,delay_sd
4881,2016-02-13 07:05:00,2016-02-13T13:05:00,82.016133,82.0,0.070188
4889,2016-02-13 06:25:00,2016-02-13T12:25:00,84.871056,84.9,0.091293
4897,2016-02-13 05:45:00,2016-02-13T11:45:00,81.850222,81.8,0.064721
4904,2016-02-13 05:10:00,2016-02-13T11:10:00,81.870083,81.9,0.052249
4913,2016-02-13 04:25:00,2016-02-13T10:25:00,82.506900,82.3,0.632347
4920,2016-02-13 03:50:00,2016-02-13T09:50:00,82.192867,82.1,0.272634
4927,2016-02-13 03:15:00,2016-02-13T09:15:00,85.208000,85.2,0.112336
4935,2016-02-13 02:35:00,2016-02-13T08:35:00,82.496433,82.5,0.078256
4941,2016-02-13 02:05:00,2016-02-13T08:05:00,81.778833,81.8,0.070960
4949,2016-02-13 01:25:00,2016-02-13T07:25:00,82.295933,82.3,0.085030


In [54]:
latency_rawdata.plot()

,datetime,delay_mean,delay_median,delay_sd
timestamp,,,,
2016-03-01 00:00:00,2016-03-01T06:00:00,80.926803,80.9,0.234238
2016-02-29 22:05:00,2016-03-01T04:05:00,82.945767,82.9,0.739154
2016-02-29 21:30:00,2016-03-01T03:30:00,86.296000,86.3,0.078000
2016-02-29 19:55:00,2016-03-01T01:55:00,77.228367,77.4,0.262130
2016-02-29 18:25:00,2016-03-01T00:25:00,81.340833,81.5,0.483414
2016-02-29 16:55:00,2016-02-29T22:55:00,81.415167,81.4,0.080125
2016-02-29 16:15:00,2016-02-29T22:15:00,81.575417,81.6,0.067298
2016-02-29 14:45:00,2016-02-29T20:45:00,82.651367,82.6,0.077748
2016-02-29 14:10:00,2016-02-29T20:10:00,82.276722,82.3,0.082039


In [52]:
latency_rawdata = latency_rawdata.set_index('timestamp')